## Notka

Jak Cię interesuje tylko rezultat - otwórz plik `partie.csv`.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
adres = "https://pl.wikipedia.org/wiki/Pos%C5%82owie_na_Sejm_Rzeczypospolitej_Polskiej_{}_kadencji"
kadencje = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII"]

In [3]:
adres.format(kadencje[0])

'https://pl.wikipedia.org/wiki/Pos%C5%82owie_na_Sejm_Rzeczypospolitej_Polskiej_I_kadencji'

In [4]:
zupa = BeautifulSoup(requests.get(adres.format(kadencje[0])).content)

In [5]:
tabela = zupa.find("table", "wikitable")

In [6]:
tabela.find("th").text

'Ugrupowanie polityczne[a]'

In [7]:
tabela.find_all("tr")[8]

<tr>
<td style="background: #FFFF00;"></td>
<td style="text-align:left;"><a href="/wiki/Niezale%C5%BCny_Samorz%C4%85dny_Zwi%C4%85zek_Zawodowy_%E2%80%9ESolidarno%C5%9B%C4%87%E2%80%9D" title="Niezależny Samorządny Związek Zawodowy „Solidarność”">Niezależny Samorządny Związek Zawodowy „Solidarność”</a></td>
<td>27</td>
<td>26</td>
<td><span style="white-space:nowrap;"><img alt="Decrease2.svg" data-file-height="300" data-file-width="300" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/11px-Decrease2.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/17px-Decrease2.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Decrease2.svg/22px-Decrease2.svg.png 2x" width="11"/> <span style="font-size: 12px; vertical-align: middle;">1</span></span></td>
</tr>

In [8]:
def parse_row(tr):
    tds = tr.find_all("td")
    kolor = tds[0].attrs["style"][-8:-1]
    a = tds[1].find("a")
    partia = a.text
    wikilink = a.attrs["href"]
    if partia.startswith("["):
        partia = "Niezrzeszeni"
        wikilink = ""
    return {"partia": partia, "kolor": kolor, "wikilink": wikilink}

In [9]:
[parse_row(tr) for tr in tabela.find_all("tr")[1:-1]]

[{'kolor': '#E2001A',
  'partia': 'Sojusz Lewicy Demokratycznej',
  'wikilink': '/wiki/Sojusz_Lewicy_Demokratycznej'},
 {'kolor': '#FF9900',
  'partia': 'Unia Demokratyczna',
  'wikilink': '/wiki/Unia_Demokratyczna'},
 {'kolor': '#1BB100',
  'partia': 'Polskie Stronnictwo Ludowe',
  'wikilink': '/wiki/Polskie_Stronnictwo_Ludowe'},
 {'kolor': '#FCA241',
  'partia': 'Polski Program Liberalny',
  'wikilink': '/wiki/Polski_Program_Liberalny'},
 {'kolor': '#333300',
  'partia': 'Konfederacja Polski Niepodległej',
  'wikilink': '/wiki/Konfederacja_Polski_Niepodleg%C5%82ej'},
 {'kolor': '#007FFF',
  'partia': 'Zjednoczenie Chrześcijańsko-Narodowe',
  'wikilink': '/wiki/Zjednoczenie_Chrze%C5%9Bcija%C5%84sko-Narodowe'},
 {'kolor': '#33CC66',
  'partia': 'Konwencja Polska',
  'wikilink': '/wiki/Konwencja_Polska'},
 {'kolor': '#FFFF00',
  'partia': 'Niezależny Samorządny Związek Zawodowy „Solidarność”',
  'wikilink': '/wiki/Niezale%C5%BCny_Samorz%C4%85dny_Zwi%C4%85zek_Zawodowy_%E2%80%9ESolidarno%

In [10]:
def fetch_and_parse_kadencja(kadencja):
    zupa = BeautifulSoup(requests.get(adres.format(kadencja)).content)
    tabela = zupa.find("table", "wikitable")
    naglowek = tabela.find("th").text
    if not naglowek.startswith("Ugrupowanie polityczne"):
        raise Exception("Pierwsza tabela nie zaczyna się od 'Ugrupowanie polityczne'")
    res = [parse_row(tr) for tr in tabela.find_all("tr")[1:-1]]
    return res

In [11]:
dfs = []
for i, kadencja in enumerate(kadencje):
    df = pd.DataFrame(fetch_and_parse_kadencja(kadencja))
    df["kadencja"] = i + 1
    dfs.append(df)

In [12]:
partie_i_kolor = pd.concat(dfs, ignore_index=True)
partie_i_kolor = partie_i_kolor[["kadencja", "partia", "kolor", "wikilink"]]

In [13]:
partie_i_kolor.head()

,kadencja,partia,kolor,wikilink
0,1,Sojusz Lewicy Demokratycznej,#E2001A,/wiki/Sojusz_Lewicy_Demokratycznej
1,1,Unia Demokratyczna,#FF9900,/wiki/Unia_Demokratyczna
2,1,Polskie Stronnictwo Ludowe,#1BB100,/wiki/Polskie_Stronnictwo_Ludowe
3,1,Polski Program Liberalny,#FCA241,/wiki/Polski_Program_Liberalny
4,1,Konfederacja Polski Niepodległej,#333300,/wiki/Konfederacja_Polski_Niepodleg%C5%82ej


In [14]:
partie_i_kolor.tail(1).iloc[0]

kadencja                                                    8
partia                               KWW Mniejszość Niemiecka
kolor                                                 #0780C4
wikilink    /wiki/Mniejszo%C5%9B%C4%87_Niemiecka_(komitet_...
Name: 91, dtype: object

In [15]:
czy_wszytkie = True
for partia, df_partii in partie_i_kolor.groupby("partia"):
    if not (df_partii["kolor"] == df_partii["kolor"].iloc[0]).all():
        print("Partia {} ma różne kolory".format(partia))
        czy_wszytkie = False
    if not (df_partii["wikilink"] == df_partii["wikilink"].iloc[0]).all():
        print("Partia {} ma różne wikilinki".format(partia))
        czy_wszytkie = False
        
if czy_wszytkie:
    print("Wszystkie partie mają kolory i wikilinki te same we wszystkich latach!")

Wszystkie partie mają kolory i wikilinki te same we wszystkich latach!


In [16]:
partie_i_kolor.to_csv("partie_per_kadencja.csv", index=False)

In [17]:
partie_skrocone = partie_i_kolor.groupby("partia").first().drop("kadencja", axis=1)

In [18]:
partie_skrocone["kadencja_pierwsza"] = partie_i_kolor.groupby("partia")["kadencja"].min()
partie_skrocone["kadencja_ostatnia"] = partie_i_kolor.groupby("partia")["kadencja"].max()

In [19]:
len(partie_skrocone)

50

In [20]:
partie_skrocone.head()

,kolor,wikilink,kadencja_pierwsza,kadencja_ostatnia
partia,,,,
Akcja Polska,#E9967A,/wiki/Akcja_Polska,1,1
Akcja Wyborcza Solidarność,#FFFF00,/wiki/Akcja_Wyborcza_Solidarno%C5%9B%C4%87,3,3
Alternatywa,#336699,/wiki/Alternatywa_Ruch_Spo%C5%82eczny,3,3
Bezpartyjny Blok Wspierania Reform,#000000,/wiki/Bezpartyjny_Blok_Wspierania_Reform,2,2
Biało-Czerwoni,#FF0004,/wiki/Bia%C5%82o-Czerwoni,7,7


In [21]:
partie_skrocone.to_csv("partie.csv")